# IMPORT

In [1]:
from __future__ import annotations
from bqskit import Circuit, compile
from bqskit.compiler.machine import MachineModel
from bqskit.ir.gates import RXXGate, RXGate, RZGate,RYGate,RZZGate,HGate,CNOTGate
from transpiler.BQSkit_custom_gate_definitions import VirtualZGate, GPIGate, GPI2Gate,PartialMSGate,FullMSGate
from transpiler.QASM2_reader import load_qasm
from bqskit.ext import bqskit_to_qiskit
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit import transpile
from qiskit.transpiler import Target, InstructionProperties
import pennylane as qml
import numpy as np

In [2]:
# Step 1: Define the MS-Gate Machine Model
IONQ_gate_set={PartialMSGate(),GPIGate(),GPI2Gate()}
def transpile_to_ms(circuit: Circuit) -> Circuit:
    """Transpile the given circuit to use only MS gates."""
    model = MachineModel(circuit.num_qudits, gate_set=IONQ_gate_set)
    compiled_circuit = compile(circuit, model, optimization_level=2)
    return compiled_circuit

# Step 2: Create a simple quantum circuit
qc = Circuit(2)
qc.append_gate(HGate(), [0])
qc.append_gate(HGate(), [1])
qc.append_gate(CNOTGate(), [1,0])
qc.append_gate(RXGate(), [1],params=[1.1561])
qc.append_gate(HGate(), [0])
qc.append_gate(RZZGate(), [0, 1],params=[1])
qc.append_gate(RYGate(), [1],params=[3])
qc.append_gate(RXXGate(), [0,1],params=[2.14584545])

    
qc_qiskit = bqskit_to_qiskit(qc)

# Step 3: Transpile the circuit to MS gates
ms_circuit = transpile_to_ms(qc)

In [3]:
#SAVE QASM
ms_circuit.save("circuit.qasm")
print(qc_qiskit.draw(output='text'))

# Initialize the latest AerSimulator
simulator = AerSimulator(method="statevector")

# Transpile the circuit for the simulator
qc_transpiled = transpile(qc_qiskit, simulator)

# Add instruction to save statevector
qc_transpiled.save_statevector()

# Run the circuit in statevector mode
result = simulator.run(qc_transpiled).result()

# Get the statevector
statevector = result.get_statevector()

# Print the statevector
np.abs(np.array(statevector))**2

     ┌───┐┌───┐    ┌───┐                      ┌──────────────┐
q_0: ┤ H ├┤ X ├────┤ H ├──────■───────────────┤0             ├
     ├───┤└─┬─┘┌───┴───┴────┐ │ZZ(1) ┌───────┐│  Rxx(2.1458) │
q_1: ┤ H ├──■──┤ Rx(1.1561) ├─■──────┤ Ry(3) ├┤1             ├
     └───┘     └────────────┘        └───────┘└──────────────┘


array([0.10533642, 0.41539816, 0.12272557, 0.35653985])

In [4]:
# Step 4: Transform to qiskit and print
qiskit_circuit = load_qasm("circuit.qasm")
print(qiskit_circuit.draw(output='text'))


-11.009299754122242
-4.726114446942656
1.55707086023693
-4.613940176445819
1.669245130733767
0.296227291124854
6.070657569746712
4.499861242951815
2.9290649161569187
1.358268589362022
3.141592686678747
     ┌──────────────┐┌─────────────┐┌──────────────────────────────────┐»
q_0: ┤ Gpi2(6.0775) ├┤ Gpi(6.1735) ├┤0                                 ├»
     ├──────────────┤└─────────────┘│  Partialms(1.5571,1.6692,1.4674) │»
q_1: ┤ Gpi2(1.9071) ├───────────────┤1                                 ├»
     └──────────────┘               └──────────────────────────────────┘»
«     ┌─────────────┐ ┌───────────────┐┌──────────────┐»
«q_0: ┤ Gpi(3.3733) ├─┤ Gpi2(0.19155) ├┤ Gpi2(4.8756) ├»
«     ├─────────────┴┐└┬──────────────┤└──────────────┘»
«q_1: ┤ Gpi(0.28936) ├─┤ Gpi2(4.0242) ├────────────────»
«     └──────────────┘ └──────────────┘                »
«     ┌───────────────────────────────────┐┌─────────────┐┌──────────────┐»
«q_0: ┤0                                  ├┤ Gpi(3.0978) ├┤ Gpi2(3.

# NOISE

In [5]:
# Convert Qiskit circuit to PennyLane
qml_circuit = qml.from_qiskit(qiskit_circuit)
np.abs(qml.matrix(qml_circuit, wire_order=[0, 1])().T[0])**2#check

array([0.10533644, 0.12272556, 0.41539816, 0.35653984])

In [6]:
@qml.qnode(qml.device("default.mixed", wires=2))  
def noisy_circuit():
    tape = qml.transforms.make_tape(qml_circuit)()

    for op in tape.operations:

        qml.QubitUnitary(op.parameters[0], wires=op.wires)  

        if len(op.wires) == 2:  # MS gates
            for qubit in op.wires:  
                qml.DepolarizingChannel(0.015, wires=qubit)  # Adjusted probability

    return qml.probs(wires=[0, 1])

noisy_result = noisy_circuit()
print("Noisy simulation result:", noisy_result)


Noisy simulation result: [0.11351723 0.12531351 0.40762914 0.35354012]
